# Understanding Hired Rides in NYC

_[Project prompt](https://docs.google.com/document/d/1uAUJGEUzfNj6OsWNAimnYCw7eKaHhMUfU1MTj9YwYw4/edit?usp=sharing), [grading rubric](https://docs.google.com/document/d/1hKuRWqFcIdhOkow3Nljcm7PXzIkoa9c_aHkMKZDxWa0/edit?usp=sharing)_

_This scaffolding notebook may be used to help setup your final project. It's **totally optional** whether you make use of this or not._

_If you do use this notebook, everything provided is optional as well - you may remove or add prose and code as you wish._

_**All code below should be consider "pseudo-code" - not functional by itself, and only an outline to help you with your own approach.**_

## Project Setup

In [318]:
# all import statements needed for the project, for example:
import math
import os
import bs4
import re
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
import pyarrow.parquet as pq
import matplotlib.animation as animation
import pandas as pd
import requests
import itertools
import sqlalchemy as db
import folium
from folium.plugins import HeatMap
import geopandas as gpd
import numpy as np
import scipy.stats as st
from geopy.distance import distance

In [319]:
"""any constants you might need; some have been added for you, and some you need to fill in"""

TAXI_URL = "https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page"

TAXI_ZONES_DIR = "data/taxi_zones"
TAXI_ZONES_SHAPEFILE = "taxi_zones.shp"
UBER_CSV = ""
WEATHER_CSV_DIR = ""

CRS = 4326  # coordinate reference system

# (lat, lon)
NEW_YORK_BOX_COORDS = ((40.560445, -74.242330), (40.908524, -73.717047))
LGA_BOX_COORDS = ((40.763589, -73.891745), (40.778865, -73.854838))
JFK_BOX_COORDS = ((40.639263, -73.795642), (40.651376, -73.766264))
EWR_BOX_COORDS = ((40.686794, -74.194028), (40.699680, -74.165205))

DATABASE_URL = "sqlite:///project.db"
DATABASE_SCHEMA_FILE = "schema.sql"
QUERY_DIRECTORY = "queries"

In [320]:
"""Make sure the QUERY_DIRECTORY exists"""
try:
    os.mkdir(QUERY_DIRECTORY)
except Exception as e:
    if e.errno == 17:
        # the directory already exists
        pass
    else:
        raise

## Part 1: Data Preprocessing

### Load Taxi Zones

In [321]:
""" This function takes the shapefile and returns an object
    consisting of each zone, locationId and its geomtry coordinates """

def load_taxi_zones(shapefile: str) -> dict:
    gdf = gpd.read_file(shapefile)

    taxi_zones = []

    for index, row in gdf.iterrows():
        zone = row.iloc[3]
        locationId = row.iloc[4]
        geometry = row.iloc[6]
        
        row_object = { "zone": zone, "locationId": locationId, "geometry": geometry }
        taxi_zones.append(row_object)
    
    return taxi_zones

In [322]:
shape_data = load_taxi_zones(TAXI_ZONES_SHAPEFILE)
# There are 263 location Ids 

In [323]:
""" This function accepts the zone id and the taxi zones
    and matches the zone id with its relevant coordinates """

def lookup_coords_for_taxi_zone_id(zone_loc_id: int, loaded_taxi_zones: list) -> int:
    for i in loaded_taxi_zones:
        if i['locationId'] == zone_loc_id:
            return i['geometry']

In [324]:
cord = []
for x in range(1,264): 
    cord.append(x)

poly = []
for x in cord:
    poly.append(lookup_coords_for_taxi_zone_id(x, shape_data))




[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 22

In [326]:
import pyproj
from shapely.geometry import Point

# Define the CRS of your input coordinates
input_crs = 'EPSG:2263'

# Define the CRS to which you want to convert your coordinates
output_crs = 'EPSG:4326'

# Create a PyProj transformer object to convert between the two CRSs
transformer = pyproj.Transformer.from_crs(input_crs, output_crs)
l = []
for i in range(len(poly)):
    if i == 56 or i == 103 or i == 104:
        l.append((None,None))
    else:
        centroid2 = Point(poly[i].centroid.x, poly[i].centroid.y)
        centroid_wgs84 = transformer.transform(centroid2.x, centroid2.y)
        l.append(centroid_wgs84)

my_dict = {index: item for index, item in enumerate(l, start=1)}


[(40.69183016020959, -74.17400156582248),
 (40.61674619937379, -73.83129979354713),
 (40.86447372906584, -73.8474217852696),
 (40.72375208451233, -73.97696827424141),
 (40.55265878064343, -74.18848459794721),
 (40.600324409468406, -74.07177024696533),
 (40.761492617043125, -73.91969433569462),
 (40.77855862576789, -73.92308626324494),
 (40.75103435668664, -73.78794875477833),
 (40.67895308442328, -73.79098676199028),
 (40.60427268170344, -74.00748784386448),
 (40.70294582144012, -74.01556349991374),
 (40.712037924667264, -74.01607927269924),
 (40.62483367229869, -74.0298925099),
 (40.7833330046879, -73.78597285841018),
 (40.76273753216493, -73.77342112932584),
 (40.69150702161197, -73.94990480444068),
 (40.86768222238494, -73.89018381164814),
 (40.73548651046706, -73.72665540152784),
 (40.857779440562844, -73.88586744911127),
 (40.60142907301775, -73.98353783537917),
 (40.61221766982795, -73.99525864587314),
 (40.60644824321443, -74.17088508711593),
 (40.80197051563418, -73.96547935663

In [7]:
""" test - lookup_coords_for_taxi_zone_id() """

def lookup_coords_for_taxi_zone_id_test():

    zones = [{ "zone": 3, "locationId": 1, "geometry": 5 }, { "zone": 8, "locationId": 7, "geometry": 3 }]
    assert lookup_coords_for_taxi_zone_id(1, zones)  == 5

### Calculate distance

In [329]:
""" This function calculate the distance giving the pick up
    point and drop off point and returns a distance integer """

def calculate_distance_with_coords(from_coord: tuple, to_coord: tuple) -> int:
    pickup_latitude, pickup_longitude = from_coord
    dropoff_latitude, dropoff_longitude = to_coord

    coords = [pickup_latitude, dropoff_latitude, pickup_longitude, dropoff_longitude]

    for i in coords:
        if i < -90 or i > 90:
            return -1

    return distance((pickup_latitude, pickup_longitude), (dropoff_latitude, dropoff_longitude)).miles

In [330]:
""" test - calculate_distance_with_coords() """

def calculate_distance_with_coords_test():

    from_coord = (37.7749, -12.4194)
    to_coord = (34.0522, -11.2437)
    assert round(calculate_distance_with_coords(from_coord, to_coord), 2) == 264.99


    from_coord = (105, -122.4194)
    to_coord = (34.0522, -118.2437) 
    assert calculate_distance_with_coords(from_coord, to_coord) == -1

In [331]:
""" This function adds a new column with the distance between coordinates to the Dataframe.
    The input is a dataframe and the output is the new dataframe """
 
def add_distance_column(dataframe: pd.DataFrame) -> pd.DataFrame:
    # Apply the calculate_distance_with_coords function to each row of the DataFrame
    distances = dataframe.apply(lambda row: calculate_distance_with_coords(
        (row["pickup_latitude"], row["pickup_longitude"]),
        (row["dropoff_latitude"], row["dropoff_longitude"])
    ), axis=1)
    
    # Add the distances as a new column to the DataFrame
    dataframe["distance"] = distances
    
    return dataframe

In [332]:
""" test - add_distance_column() """

def add_distance_column_test():

    df = pd.DataFrame({
        'pickup_latitude': [40.7128],
        'pickup_longitude': [-74.006],
        'dropoff_latitude': [40.7851],
        'dropoff_longitude': [-73.9683]
    })

    df_with_distance = add_distance_column(df)

    assert "distance" in df_with_distance.columns, 'distance column is not present'

### Process Taxi Data

In [ ]:
""" This function downloads all the relevant files from the taxi webpage
    and places it into our local directory """

def download_files(month: int, year: int):
    formatted_month = f"{month:02d}"
    current_dir = os.getcwd()
    url = f"https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{year}-{formatted_month}.parquet"
    
    windows = f"{current_dir}\\"
    str = windows if  os.name == 'nt' else ""

    response = requests.get(url, stream=True)
    with open(f"{str}yellow_taxi_{year}_{formatted_month}.parquet", "wb") as f:
        for chunk in response.iter_content(chunk_size=1024): 
            if chunk:
                f.write(chunk)

years = list(range(2009, 2016))
months = list(range(1, 13))

for year in years:
    if year < 2015:
        for month in months:
            download_files(month, year)
    else:
        for month in range(1, 7):
            download_files(month, year)

In [333]:
""" This function gets all the URLs from the taxi web page and returns
    it as an array of strings """

def get_all_urls_from_taxi_page(taxi_page: str) -> list[str]:
    try:
        response = requests.get(taxi_page)

        soup = bs4.BeautifulSoup(response.content, 'html.parser')
        urls = []

        for link in soup.find_all('a'):
            href = link.get('href')
            if href is not None:
                urls.append(href)

        return urls
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

In [334]:
"""" test for get_all_urls_from_taxi_page() """

def get_all_urls_from_taxi_page_test():

    assert len(get_all_urls_from_taxi_page(TAXI_URL)) == 483

In [335]:
""" This function goes through all the URLs on the taxi web page
    and returns only the ones ending in .parquet since we want
    parquet files and also the ones from the years 2009 to 2015
    to avoid iterating through unecessary files. """

def filter_taxi_parquet_urls(all_urls: list[str]) -> list[str]:
    parquet_urls = []
    years = list(range(2009, 2016))

    if all_urls is not None:
        for i in all_urls:
            str = re.search('.parquet$', i)

            if(str != None and "yellow_tripdata" in i):
                year = int(i.split("_")[2][:4])

                if year in years:
                    parquet_urls.append(i)
    return parquet_urls

In [336]:
""" test for filter_taxi_parquet_urls() """

def filter_taxi_parquet_urls_test():

    allUrlsData = get_all_urls_from_taxi_page(TAXI_URL)
    assert len(filter_taxi_parquet_urls(allUrlsData)) == 84

In [337]:
""" This function takes a URL and extracts the month from it
    The example url can look like:
    https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-06.parquet """

def get_and_clean_month(url: str) -> str:
    str = url[len(url) - 10:]
    [month, fileType] = str.split('.')
    return month

In [338]:
""" test for get_and_clean_month function """

def get_and_clean_month_test():

    url = 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-06.parquet'
    assert get_and_clean_month(url) == '06'

In [339]:
""" This function takes a URL and extracts the year from it
    The example url can look like:
    https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-06.parquet """

def get_and_clean_year(url: str) -> str:
    str = url[len(url) - 15:]
    [year, other] = str.split('-')
    return year

In [340]:
""" test for get_and_clean_year function """

def get_and_clean_year_test():

    url = 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-06.parquet'
    assert get_and_clean_year(url) == '2022'

In [341]:
""" This fucntion adds a new column with the distance between coordinates to the taxi Dataframe.
    The input is a dataframe and the output is the new modified dataframe """
 
def add_distance_column_taxi(dataframe: pd.DataFrame) -> pd.DataFrame:
    # Apply the calculate_distance_with_coords function to each row of the DataFrame
    distances = dataframe.apply(lambda row: calculate_distance_with_coords(
        (row["Start_Lat"], row["Start_Lon"]),
        (row["End_Lat"], row["End_Lon"])
    ), axis=1)
    
    # Add the distances as a new column to the DataFrame
    dataframe["distance"] = distances
    
    return dataframe["distance"]

In [342]:
""" test - add_distance_column_taxi() """

def add_distance_column_taxi_test():

    df_taxi = pd.DataFrame({
        'Start_Lat': [40.7128],
        'Start_Lon': [-74.006],
        'End_Lat': [40.7851],
        'End_Lon': [-73.9683]
    })

    df_taxi_with_distance = add_distance_column_taxi(df_taxi)

    assert df_taxi_with_distance.shape[0] == 1
    assert isinstance(df_taxi_with_distance, pd.Series)

In [355]:
""" This function collects all the parquet urls from the taxi website.
    It will then get the actual data from the parquet files and do various forms of cleaning.
    For example, we will remove unnecessary columns and invalid data and will return
    one gigantic dataframe with data from every month """

def convert_taxi_data(parquet_urls: list[str]) -> pd.DataFrame:
    all_taxi_dataframes = []
    
    for parquet_url in parquet_urls:
        month = get_and_clean_month(parquet_url)
        year = get_and_clean_year(parquet_url)

        cwd = os.getcwd()
        files = os.listdir(cwd)

        fileName = f"yellow_taxi_{year}_{month}.parquet"
        if fileName in files :

            dataframe = pd.read_parquet(fileName)
            sample_dataframe = dataframe.sample(n=3000)
            all_taxi_dataframes.append(sample_dataframe)
        
    # create one gigantic dataframe with data from every month needed
    taxi_data = pd.concat(all_taxi_dataframes)
    return taxi_data

In [356]:
""" This function gets all the urls from the taxi page, specifically the parquet urls,
    gets and cleans it, and returns the valid data """

def get_taxi_data() -> pd.DataFrame:
    all_urls = get_all_urls_from_taxi_page(TAXI_URL)
    all_parquet_urls = filter_taxi_parquet_urls(all_urls)
    taxi_data = convert_taxi_data(all_parquet_urls)

    return taxi_data

In [357]:
"""This fucntion selects only the needed columns and dropps the rest. It also combines some columns as the data is spread across mutltiple
"""

def set_up_taxi_dataset() -> pd.DataFrame:

    selected_cols = ['tpep_pickup_datetime', 'tpep_dropoff_datetime', 'passenger_count', 'trip_distance','PULocationID', 'DOLocationID','fare_amount','tip_amount','total_amount','pickup_datetime', 'dropoff_datetime', 'pickup_longitude', 'pickup_latitude',  'dropoff_longitude', 'dropoff_latitude', 'Trip_Pickup_DateTime', 'Trip_Dropoff_DateTime', 'Passenger_Count', 'Trip_Distance', 'Start_Lon', 'Start_Lat',  'End_Lon', 'End_Lat',  'Fare_Amt', 'Tip_Amt', 'Total_Amt']
    df_selected = get_taxi_data()
    df_selected = df_selected[selected_cols]
    # list of column pairs to join
    column_pairs = [("tpep_pickup_datetime", 'pickup_datetime'), 
                    ("tpep_dropoff_datetime", 'dropoff_datetime'),
                    ('Trip_Distance', 'trip_distance'),
                    ('Passenger_Count', 'passenger_count'),
                    ('Start_Lon', 'PULocationID'),
                    ('Start_Lat', 'PULocationID'),
                    ('End_Lon', 'DOLocationID'),
                    ('End_Lat', 'DOLocationID'),
                    ('Fare_Amt', 'fare_amount'),
                    ('Tip_Amt', 'tip_amount'),
                    ('Total_Amt', 'total_amount')]

    # loop over column pairs and join them
    for pair in column_pairs:
        # fill missing values in the first column with values from the second column
        df_selected[pair[0]] = df_selected[pair[0]].fillna(df_selected[pair[1]])
        # drop the second column
        df_selected_final = df_selected.drop(pair[1], axis=1)

    df_selected_final = df_selected_final.drop(['pickup_datetime',	'dropoff_datetime', 'passenger_count', 'trip_distance', 'PULocationID', 'DOLocationID', 'fare_amount', 'tip_amount', 'pickup_longitude',	'pickup_latitude',	'dropoff_longitude'	,'dropoff_latitude'], axis=1)


    column_pairs = [("tpep_pickup_datetime", 'Trip_Pickup_DateTime'), 
                    ("tpep_dropoff_datetime", 'Trip_Dropoff_DateTime')]


    # loop over column pairs and join them
    for pair in column_pairs:
        # fill missing values in the first column with values from the second column
        df_selected_final[pair[0]] = df_selected_final[pair[0]].fillna(df_selected_final[pair[1]])
        # drop the second column
        df_selected_final = df_selected_final.drop(pair[1], axis=1)

        return df_selected_final



In [358]:
DF= set_up_taxi_dataset()

In [359]:
df2 = DF.copy()

In [360]:
# define a function that performs the lookup operation
def lookup_lat(x):
    result = my_dict.get(x)
    if result is not None:
        return result[0]
    else:
        return x
    
# define a function that performs the lookup operation
def lookup_lon(x):
    result = my_dict.get(x)
    if result is not None:
        return result[1]
    else:
        return x

In [361]:
# apply the lookup function to all the columns in the DataFrame
#%time
df2["End_Lon"] = df2["End_Lon"].apply(lambda x: lookup_lon(x))
df2["Start_Lon"] = df2["Start_Lon"].apply(lambda x: lookup_lon(x))

df2["Start_Lat"] = df2["Start_Lat"].apply(lambda x: lookup_lat(x))
df2["End_Lat"] = df2["End_Lat"].apply(lambda x: lookup_lat(x))

In [363]:
'''
We then filter based on coordinates to make sure the rides are within the coordinates we want.
We also remove trips with 0 passangers and no fares. We further remove trips with passangers above 6 as that 
is uber policy. Lastly we remove trips with no distace between dropoff and pickup. The output is the
cleaned dataframe
'''
def filter_taxi_data() -> pd.DataFrame:

    df_selected_final = df2
    df_selected_final = df_selected_final[(df_selected_final["Start_Lat"] >= 40.560445) & 
                                        (df_selected_final["Start_Lon"] >= -74.242330) & 
                                        (df_selected_final["Start_Lat"] <= 40.908524) & 
                                        (df_selected_final["Start_Lon"] <= -73.717047) &
                                        (df_selected_final["End_Lat"] >= 40.560445) & 
                                        (df_selected_final["End_Lon"] >= -74.242330) & 
                                        (df_selected_final["End_Lat"] <= 40.908524) & 
                                        (df_selected_final["End_Lon"] <= -73.717047)]


    df_selected_final = df_selected_final[df_selected_final['Passenger_Count'] != 0]

    add_distance_column_taxi(df_selected_final)

    df_selected_final = df_selected_final.drop(index=df_selected_final[df_selected_final['distance'] == 0].index)

    df_selected_final = df_selected_final[df_selected_final['Passenger_Count']<=6.0]
    df_selected_final = df_selected_final.reset_index(drop=True)
    df_selected_final = df_selected_final.rename(columns={
        "tpep_pickup_datetime": "pickup_datetime",
        "tpep_dropoff_datetime": "dropoff_datetime",
        "Passenger_Count": "Passenger_Count",
        "Trip_Distance": "Trip_Distance",
        "Start_Lon": "Start_Lon",
        "Start_Lat": "Start_Lat",
        "End_Lon": "End_Lon",
        "End_Lat": "End_Lat",
        "Fare_Amt": "Fare_Amt",
        "Tip_Amt": "Tip_Amt",
        "Total_Amt": "Total_Amt",
        "distance": "distance"


    })
    df_selected_final = df_selected_final.drop('dropoff_datetime', axis=1)
    df_selected_final.rename(columns={"Trip_Dropoff_DateTime": "dropoff_datetime"})
    return df_selected_final


In [364]:
Taxi_Data = filter_taxi_data()



In [365]:
Taxi_Data




,pickup_datetime,Trip_Dropoff_DateTime,Passenger_Count,Trip_Distance,Start_Lon,Start_Lat,End_Lon,End_Lat,Fare_Amt,Tip_Amt,Total_Amt,distance
0,2015-01-12 21:11:01,NaN,1.0,1.00,-73.989845,40.762253,-73.999918,40.748427,5.0,0.00,6.30,1.090615
1,2015-01-14 01:49:41,NaN,2.0,1.48,-73.989845,40.762253,-73.990458,40.740337,7.0,0.00,8.30,1.512552
2,2015-01-17 19:20:20,NaN,5.0,2.89,-73.985937,40.727620,-74.016079,40.712038,12.0,2.40,15.20,1.913164
3,2015-01-28 08:39:20,NaN,5.0,1.47,-73.990458,40.740337,-74.007486,40.726290,7.5,2.08,10.38,1.318463
4,2015-01-30 09:48:52,NaN,2.0,0.50,-73.985937,40.727620,-73.997380,40.728340,5.0,0.00,5.80,0.602720
...,...,...,...,...,...,...,...,...,...,...,...,...
183441,2009-12-07 18:20:00,2009-12-07 18:30:47,1.0,1.80,-73.973607,40.764044,-73.952651,40.778368,8.1,0.00,9.60,1.478338
183442,2009-12-16 06:53:14,2009-12-16 07:01:15,3.0,1.70,-73.978104,40.752521,-73.976706,40.739227,6.9,0.00,7.40,0.920250
183443,2009-12-13 20:43:54,2009-12-13 20:47:29,1.0,0.50,-73.957094,40.775995,-73.951589,40.773130,4.1,0.00,5.10,0.349963
183444,2009-12-15 20:34:11,2009-12-15 20:37:20,1.0,0.70,-73.974285,40.753180,-73.972398,40.746405,4.1,1.00,6.10,0.477866


### Processing Uber Data

In [366]:
"""This function first loads the uber data from the csv file. 
We then filter based on coordinates to make sure the rides are within the coordinates we want.
We also remove trips with 0 passangers and no fares. We further remove trips with passangers above 6 as that 
is uber policy. Lastly we remove trips with no distace between dropoff and pickup. The output is the
cleaned dataframe"""

def load_and_clean_uber_data(csv_file: str) -> pd.DataFrame:

    # Reading in file into a data frame 
    uber_data = pd.read_csv(csv_file)

    # Filter data based on pickup and dropoff latitude/longitude(40.560445, -74.242330) and (40.908524, -73.717047).

    uber_data = uber_data[(uber_data["pickup_latitude"] >= 40.560445) & 
                      (uber_data["pickup_longitude"] >= -74.242330) & 
                      (uber_data["pickup_latitude"] <= 40.908524) & 
                      (uber_data["pickup_longitude"] <= -73.717047) &
                      (uber_data["dropoff_latitude"] >= 40.560445) & 
                      (uber_data["dropoff_longitude"] >= -74.242330) & 
                      (uber_data["dropoff_latitude"] <= 40.908524) & 
                      (uber_data["dropoff_longitude"] <= -73.717047)]
    
    # Checking if there are any null values for pickup_longitude, pickup_latitude, dropoff_longitude, dropoff_latitude
    null_drop_lat = uber_data[uber_data['dropoff_latitude'].isnull()]
    null_drop_long = uber_data[uber_data['dropoff_longitude'].isnull()]
    null_pick_lat= uber_data[uber_data['pickup_latitude'].isnull()]
    null_pick_long = uber_data[uber_data['pickup_longitude'].isnull()]

    # Return True, if none of the colums have null values 

   # if null_drop_lat.empty & null_drop_long.empty & null_pick_lat.empty & null_pick_long.empty :
        #print(True)
    #else:
       # print(False)

    
    # Removing rows where passamger count is 0 
    uber_data = uber_data[uber_data['passenger_count']!=0]


    # Removing rows with passanger data is abnormally large 
    uber_data = uber_data[uber_data['passenger_count']<=6]

    # Checking datatypes for all columns 
    #print(uber_data.dtypes)

    #Making sure pickup time is a datetime object and normalizing the name 
    uber_data ['pickup_time'] = pd.to_datetime(uber_data ['pickup_datetime'])
 
    return uber_data

In [367]:
load_and_clean_uber_data("uber_rides_sample.csv")

,Unnamed: 0,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,pickup_time
0,24238194,2015-05-07 19:52:06.0000003,7.5,2015-05-07 19:52:06 UTC,-73.999817,40.738354,-73.999512,40.723217,1,2015-05-07 19:52:06+00:00
1,27835199,2009-07-17 20:04:56.0000002,7.7,2009-07-17 20:04:56 UTC,-73.994355,40.728225,-73.994710,40.750325,1,2009-07-17 20:04:56+00:00
2,44984355,2009-08-24 21:45:00.00000061,12.9,2009-08-24 21:45:00 UTC,-74.005043,40.740770,-73.962565,40.772647,1,2009-08-24 21:45:00+00:00
3,25894730,2009-06-26 08:22:21.0000001,5.3,2009-06-26 08:22:21 UTC,-73.976124,40.790844,-73.965316,40.803349,3,2009-06-26 08:22:21+00:00
4,17610152,2014-08-28 17:47:00.000000188,16.0,2014-08-28 17:47:00 UTC,-73.925023,40.744085,-73.973082,40.761247,5,2014-08-28 17:47:00+00:00
...,...,...,...,...,...,...,...,...,...,...
199995,42598914,2012-10-28 10:49:00.00000053,3.0,2012-10-28 10:49:00 UTC,-73.987042,40.739367,-73.986525,40.740297,1,2012-10-28 10:49:00+00:00
199996,16382965,2014-03-14 01:09:00.0000008,7.5,2014-03-14 01:09:00 UTC,-73.984722,40.736837,-74.006672,40.739620,1,2014-03-14 01:09:00+00:00
199997,27804658,2009-06-29 00:42:00.00000078,30.9,2009-06-29 00:42:00 UTC,-73.986017,40.756487,-73.858957,40.692588,2,2009-06-29 00:42:00+00:00
199998,20259894,2015-05-20 14:56:25.0000004,14.5,2015-05-20 14:56:25 UTC,-73.997124,40.725452,-73.983215,40.695415,1,2015-05-20 14:56:25+00:00


In [368]:
""" We use the add distance column fcuntion we had defined before to add a new column with the distance 
of the ride to our uber data. We also drop columns where the distance of the ride is ==0"""

def get_uber_data() -> pd.DataFrame:
    uber_dataframe = load_and_clean_uber_data("uber_rides_sample.csv")
    add_distance_column(uber_dataframe)
    uber_dataframe = uber_dataframe.drop(index=uber_dataframe[uber_dataframe['distance'] == 0].index)
    return uber_dataframe

In [369]:
final_uber_data = get_uber_data()

In [370]:
#Removing unnecessary columns 
final_uber_data = final_uber_data.drop('Unnamed: 0', axis=1)
final_uber_data = final_uber_data.drop('key', axis=1)
final_uber_data = final_uber_data.drop('pickup_datetime', axis=1)

In [371]:
final_uber_data



,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,pickup_time,distance
0,7.5,-73.999817,40.738354,-73.999512,40.723217,1,2015-05-07 19:52:06+00:00,1.044594
1,7.7,-73.994355,40.728225,-73.994710,40.750325,1,2009-07-17 20:04:56+00:00,1.525071
2,12.9,-74.005043,40.740770,-73.962565,40.772647,1,2009-08-24 21:45:00+00:00,3.131464
3,5.3,-73.976124,40.790844,-73.965316,40.803349,3,2009-06-26 08:22:21+00:00,1.032372
4,16.0,-73.925023,40.744085,-73.973082,40.761247,5,2014-08-28 17:47:00+00:00,2.786061
...,...,...,...,...,...,...,...,...
199995,3.0,-73.987042,40.739367,-73.986525,40.740297,1,2012-10-28 10:49:00+00:00,0.069673
199996,7.5,-73.984722,40.736837,-74.006672,40.739620,1,2014-03-14 01:09:00+00:00,1.167951
199997,30.9,-73.986017,40.756487,-73.858957,40.692588,2,2009-06-29 00:42:00+00:00,7.995752
199998,14.5,-73.997124,40.725452,-73.983215,40.695415,1,2015-05-20 14:56:25+00:00,2.197512


### Processing Weather Data

In [372]:
"""This function takes all the weather files, iterates through them and merges them 
into one dataframe. The output is the combined dataframe"""

def get_all_weather_csvs() -> pd.DataFrame:
    years = list(range(2009, 2016))

    # Initialize an empty list to store the dataframes
    dataframes = []

    # Iterate over the weather files
    for year in years:
        filepath = f"{year}_weather.csv"
        df = pd.read_csv(filepath)
        dataframes.append(df)

    # Concatenate all the dataframes into a single dataframe
    merged_df = pd.concat(dataframes, ignore_index=True)
    return merged_df

In [373]:
"""This function first loads the uber data from the csv file. 
We then filter based on coordinates to make sure the rides are within the coordinates we want.
We also remove trips with 0 passangers and no fares. We further remove trips with passangers above 6 as that 
is uber policy. Lastly we remove trips with no distace between dropoff and pickup. The output is the
cleaned dataframe"""

def load_and_clean_weather_data() -> pd.DataFrame:

    df = get_all_weather_csvs()

    df1 = df[['STATION', 'DATE', 'LATITUDE', 'LONGITUDE', 'NAME','HourlyPrecipitation','HourlyWindGustSpeed', 'HourlyWindSpeed', 'DailyAverageWindSpeed','DailyPrecipitation']]
    df2 = df1.dropna(subset=['HourlyPrecipitation', 'HourlyWindGustSpeed'])

    #column_types = df2.dtypes

    #print(column_types)

    # we see that the averages for wind speed and precipitation are null for all values so we can drop the columns 

    # We also doing need the hourly wind gust speed as we will be using the hourly wind speed, we can drop that column as well

    df2 = df2.drop(columns=['DailyAverageWindSpeed','DailyPrecipitation', 'HourlyWindGustSpeed','LATITUDE', 'LONGITUDE'])
    df2['DATE'] = pd.to_datetime(df['DATE'])

    df2

    # Removing all rows where Hourly preicipitation has the value "T" as we do not need to measure trace amounts 

    df3 = df2[df2['HourlyPrecipitation'] != "T"]

    df4 = df3.drop(columns=["STATION"])

    df4 = df4.reset_index()

    df4['DATE'] = df4['DATE'].apply(lambda x: x.to_pydatetime())

    df4['DATE'] = pd.to_datetime(df4['DATE'])

    df4['HourlyPrecipitation'] = df4['HourlyPrecipitation'].str.replace(r'(\d+)\s*[sS]$', r'\1', regex=True)
    
    # convert column "A" from object to float
    df4['HourlyPrecipitation'] = df4['HourlyPrecipitation'].astype(float)

    Weather_Data = df4.drop('index', axis=1)

    return  Weather_Data

In [374]:
load_and_clean_weather_data()

/var/folders/4y/mgwhf9fd22746wblcfjds5gc0000gn/T/ipykernel_74492/1094114271.py:13: DtypeWarning: Columns (9,13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filepath)
/var/folders/4y/mgwhf9fd22746wblcfjds5gc0000gn/T/ipykernel_74492/1094114271.py:13: DtypeWarning: Columns (8,9,10,17) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filepath)
/var/folders/4y/mgwhf9fd22746wblcfjds5gc0000gn/T/ipykernel_74492/1094114271.py:13: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filepath)
/var/folders/4y/mgwhf9fd22746wblcfjds5gc0000gn/T/ipykernel_74492/1094114271.py:13: DtypeWarning: Columns (7,8,9,10,17,18,42,65) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filepath)
/var/folders/4y/mgwhf9fd22746wblcfjds5gc0000gn/T/ipykernel_74492/1094114271.py:13: DtypeWarning: Columns (17,78) have mix

,DATE,NAME,HourlyPrecipitation,HourlyWindSpeed
0,2009-01-06 20:00:00,"NY CITY CENTRAL PARK, NY US",0.01,10.0
1,2009-01-06 23:38:00,"NY CITY CENTRAL PARK, NY US",0.02,11.0
2,2009-01-07 02:51:00,"NY CITY CENTRAL PARK, NY US",0.09,13.0
3,2009-01-07 03:51:00,"NY CITY CENTRAL PARK, NY US",0.06,15.0
4,2009-01-07 04:51:00,"NY CITY CENTRAL PARK, NY US",0.07,16.0
...,...,...,...,...
7098,2015-12-29 10:51:00,"NY CITY CENTRAL PARK, NY US",0.02,10.0
7099,2015-12-29 11:33:00,"NY CITY CENTRAL PARK, NY US",0.02,8.0
7100,2015-12-29 11:51:00,"NY CITY CENTRAL PARK, NY US",0.02,6.0
7101,2015-12-31 11:51:00,"NY CITY CENTRAL PARK, NY US",0.00,9.0


In [375]:
"""Roll up the data to daily"""
def clean_month_weather_data_daily() -> pd.DataFrame:

    daily_data = load_and_clean_weather_data()

    daily_data_final = daily_data.groupby([daily_data['DATE'].dt.year, daily_data['DATE'].dt.month, daily_data['DATE'].dt.day]).sum()[['HourlyPrecipitation', "HourlyWindSpeed" ]]

    daily_data_final = daily_data_final.rename_axis(index=['Year', 'Month', 'Day'])
    
    return daily_data_final

In [376]:
def clean_month_weather_data_hourly() -> pd.DataFrame:

    hourly_data = load_and_clean_weather_data()

    hourly_data_final = hourly_data.groupby([hourly_data['DATE'].dt.year, hourly_data['DATE'].dt.month, hourly_data['DATE'].dt.day, hourly_data['DATE'].dt.hour]).sum()[['HourlyPrecipitation', "HourlyWindSpeed" ]]

    hourly_data_final = hourly_data_final.rename_axis(index=['Year', 'Month', 'Day', 'Hour'])
    
    return hourly_data_final

In [377]:
#hourly_weather_data, daily_weather_data = load_and_clean_weather_data()

In [378]:
hourly_weather_data = clean_month_weather_data_hourly()
hourly_weather_data.head()

/var/folders/4y/mgwhf9fd22746wblcfjds5gc0000gn/T/ipykernel_74492/1094114271.py:13: DtypeWarning: Columns (9,13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filepath)
/var/folders/4y/mgwhf9fd22746wblcfjds5gc0000gn/T/ipykernel_74492/1094114271.py:13: DtypeWarning: Columns (8,9,10,17) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filepath)
/var/folders/4y/mgwhf9fd22746wblcfjds5gc0000gn/T/ipykernel_74492/1094114271.py:13: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filepath)
/var/folders/4y/mgwhf9fd22746wblcfjds5gc0000gn/T/ipykernel_74492/1094114271.py:13: DtypeWarning: Columns (7,8,9,10,17,18,42,65) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filepath)
/var/folders/4y/mgwhf9fd22746wblcfjds5gc0000gn/T/ipykernel_74492/1094114271.py:13: DtypeWarning: Columns (17,78) have mix

HourlyPrecipitation  HourlyWindSpeed
Year Month Day Hour                                      
2009 1     6   20                   0.01             10.0
               23                   0.02             11.0
           7   2                    0.09             13.0
               3                    0.06             15.0
               4                    0.07             16.0

In [379]:
daily_weather_data = clean_month_weather_data_daily()
daily_weather_data = daily_weather_data.reset_index()

daily_weather_data

/var/folders/4y/mgwhf9fd22746wblcfjds5gc0000gn/T/ipykernel_74492/1094114271.py:13: DtypeWarning: Columns (9,13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filepath)
/var/folders/4y/mgwhf9fd22746wblcfjds5gc0000gn/T/ipykernel_74492/1094114271.py:13: DtypeWarning: Columns (8,9,10,17) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filepath)
/var/folders/4y/mgwhf9fd22746wblcfjds5gc0000gn/T/ipykernel_74492/1094114271.py:13: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filepath)
/var/folders/4y/mgwhf9fd22746wblcfjds5gc0000gn/T/ipykernel_74492/1094114271.py:13: DtypeWarning: Columns (7,8,9,10,17,18,42,65) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filepath)
/var/folders/4y/mgwhf9fd22746wblcfjds5gc0000gn/T/ipykernel_74492/1094114271.py:13: DtypeWarning: Columns (17,78) have mix

,Year,Month,Day,HourlyPrecipitation,HourlyWindSpeed
0,2009,1,6,0.03,21.0
1,2009,1,7,1.13,224.0
2,2009,1,10,0.06,48.0
3,2009,1,11,0.26,67.0
4,2009,1,17,0.69,7.0
...,...,...,...,...,...
1026,2015,12,26,0.00,76.0
1027,2015,12,27,0.02,58.0
1028,2015,12,28,0.00,70.0
1029,2015,12,29,0.75,167.0


## Part 2: Storing Cleaned Data

In [380]:
engine = db.create_engine(DATABASE_URL)

In [381]:
# if using SQL (as opposed to SQLAlchemy), define the commands 
# to create your 4 tables/dataframes
HOURLY_WEATHER_SCHEMA = """
    CREATE TABLE IF NOT EXISTS HOURLY_WEATHER (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        year INTEGER,
        month INTEGER,
        day INTEGER,
        hour INTEGER,
        precipitation REAL,
        wind REAL
);
"""

DAILY_WEATHER_SCHEMA = """
    CREATE TABLE IF NOT EXISTS DAILY_WEATHER (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        year INTEGER,
        month INTEGER,
        day INTEGER,
        precipitation REAL,
        wind REAL
    );
"""

TAXI_TRIPS_SCHEMA = """
    CREATE TABLE IF NOT EXISTS TAXI_TRIPS (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        pickup_datetime DATETIME,
        Trip_Dropoff_DateTime DATETIME
        Passenger_Count REAL
        Trip_Distance REAL,
        Start_Lon REAL,
        Start_Lat REAL,
        End_Lon REAL, 
        End_Lat REAL,
        Fare_Amt REAL, 
        Tip_Amt REAL, 
        Total_Amt REAL,
        distance REAL,

    );
"""

UBER_TRIPS_SCHEMA = """
    CREATE TABLE IF NOT EXISTS UBER_TRIPS (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        fare_amount REAL,
        pickup_longitude REAL,
        pickup_latitude REAL,
        dropoff_longitude REAL,
        dropoff_latitude REAL,
        passenger_count INTEGER,
        pickup_time DATETIME,
        distance REAL,
    );
"""


In [382]:
# create that required schema.sql file
with open(DATABASE_SCHEMA_FILE, "w") as f:
    f.write(HOURLY_WEATHER_SCHEMA)
    f.write(DAILY_WEATHER_SCHEMA)
    f.write(TAXI_TRIPS_SCHEMA)
    f.write(UBER_TRIPS_SCHEMA)

In [383]:
# create the tables with the schema files
with engine.connect() as connection:
    pass

### Add Data to Database

In [384]:
def write_dataframes_to_table():

    hourly_weather_data.to_sql(name='HOURLY_WEATHER', con=engine, if_exists='replace', index=False)
    daily_weather_data.to_sql(name='DAILY_WEATHER', con=engine, if_exists='replace', index=False)
    final_uber_data.to_sql(name='UBER_TRIPS', con=engine, if_exists='replace', index=False)
    Taxi_Data.to_sql(name='TAXI_TRIPS', con=engine, if_exists='replace', index=False)

write_dataframes_to_table()

In [385]:
from sqlalchemy import create_engine

# establish a connection to the SQL database
engine = create_engine(DATABASE_URL)

# execute a SELECT query on the HOURLY_WEATHER table
query = "SELECT * FROM TAXI_TRIPS LIMIT 10;"
result = engine.execute(query)

for row in result:
    print(row)

('2015-01-12 21:11:01.000000', None, 1.0, 1.0, -73.98984489812494, 40.76225259858687, -73.99991779023728, 40.7484273300986, 5.0, 0.0, 6.3, 1.0906149158476206)
('2015-01-14 01:49:41.000000', None, 2.0, 1.48, -73.98984489812494, 40.76225259858687, -73.9904579133977, 40.74033737258096, 7.0, 0.0, 8.3, 1.5125521768123307)
('2015-01-17 19:20:20.000000', None, 5.0, 2.89, -73.98593745737395, 40.727620122038296, -74.01607927269924, 40.712037924667264, 12.0, 2.4, 15.2, 1.9131644830367174)
('2015-01-28 08:39:20.000000', None, 5.0, 1.47, -73.9904579133977, 40.74033737258096, -74.0074858070925, 40.72629040399525, 7.5, 2.08, 10.38, 1.3184625098943075)
('2015-01-30 09:48:52.000000', None, 2.0, 0.5, -73.98593745737395, 40.727620122038296, -73.9973801630823, 40.72834036831643, 5.0, 0.0, 5.8, 0.6027204295949005)
('2015-01-21 09:48:39.000000', None, 1.0, 1.1, -73.99991779023728, 40.7484273300986, -73.9904579133977, 40.74033737258096, 7.5, 2.0, 10.3, 0.7470553727411682)
('2015-01-28 22:02:40.000000', None

## Part 3: Understanding the Data

In [386]:
# Helper function to write the queries to file
def write_query_to_file(query, outfile):
    raise NotImplementedError()

### Query 1

In [387]:
from sqlalchemy import create_engine

# establish a connection to the SQL database
engine = create_engine(DATABASE_URL)

# execute a SELECT query on the HOURLY_WEATHER table

with open('1_hour_day.sql', 'r') as file:
    query = file.read()

result = engine.execute(query)

for row in result:
    print(row)

('19', 11319)
('18', 10938)
('20', 10792)
('21', 10693)
('22', 10330)
('23', 9228)
('14', 9183)
('17', 9134)
('12', 8974)
('13', 8891)
('15', 8866)
('09', 8621)
('11', 8466)
('08', 8315)
('10', 8314)
('16', 7518)
('00', 7417)
('07', 6570)
('01', 5388)
('06', 3849)
('02', 3840)
('03', 2826)
('04', 2161)
('05', 1813)


### Query 2

In [388]:
from sqlalchemy import create_engine

# establish a connection to the SQL database

engine = create_engine(DATABASE_URL)

# execute a SELECT query on the HOURLY_WEATHER table

with open('2_day_week.sql', 'r') as file:
    
    query = file.read()

result = engine.execute(query)


for row in result:
    print(row)

('Sunday', 25503)
('Monday', 24280)
('Tuesday', 27152)
('Wednesday', 27971)
('Thursday', 28960)
('Friday', 29749)
('Saturday', 29214)


### Query 3

In [389]:
from sqlalchemy import create_engine

# establish a connection to the SQL database
engine = create_engine(DATABASE_URL)

# execute a SELECT query on the HOURLY_WEATHER table

with open('3_95_percentile.sql', 'r') as file:
    query = file.read()

result = engine.execute(query)

for row in result:
    print(row)

(16.7641931513008,)


### Query 4

In [390]:
from sqlalchemy import create_engine

# establish a connection to the SQL database
engine = create_engine(DATABASE_URL)

# execute a SELECT query on the HOURLY_WEATHER table

with open('4_top_10_days.sql', 'r') as file:
    query = file.read()

result = engine.execute(query)

for row in result:
    print(row)

OperationalError: (sqlite3.OperationalError) no such column: pickup_datetime
[SQL: SELECT pickup_date, COUNT(*) AS frequency, AVG(distance) AS avg_distance
FROM (
  SELECT date(pickup_datetime) AS pickup_date, distance
  FROM UBER_TRIPS
  WHERE pickup_datetime BETWEEN '2009-01-01' AND '2009-12-31'
  UNION ALL
  SELECT date(pickup_datetime) AS pickup_date, distance
  FROM TAXI_TRIPS
  WHERE pickup_datetime BETWEEN '2009-01-01' AND '2009-12-31'
)
GROUP BY pickup_date
ORDER BY frequency DESC
LIMIT 10;]
(Background on this error at: https://sqlalche.me/e/14/e3q8)

### Query 5

In [391]:
from sqlalchemy import create_engine

# establish a connection to the SQL database
engine = create_engine(DATABASE_URL)

# execute a SELECT query on the HOURLY_WEATHER table

with open('5_10_windiest_days.sql', 'r') as file:
    query = file.read()

result = engine.execute(query)

for row in result:
    print(row)

OperationalError: (sqlite3.OperationalError) no such column: pickup_datetime
[SQL: SELECT t.pickup_date, COUNT(*) AS frequency, AVG(w.wind) AS avg_wind
FROM (
  SELECT date(pickup_datetime) AS pickup_date
  FROM UBER_TRIPS
  WHERE pickup_datetime BETWEEN '2014-01-01' AND '2014-12-31'
  UNION ALL
  SELECT date(pickup_datetime) AS pickup_date
  FROM TAXI_TRIPS
  WHERE pickup_datetime BETWEEN '2014-01-01' AND '2014-12-31'
) t
JOIN DAILY_WEATHER w
ON t.pickup_date = date(w.year || '-' || w.month || '-' || w.day)
GROUP BY t.pickup_date
ORDER BY avg_wind DESC
LIMIT 10;]
(Background on this error at: https://sqlalche.me/e/14/e3q8)

### Query 6

In [61]:
from sqlalchemy import create_engine

# establish a connection to the SQL database
engine = create_engine(DATABASE_URL)

# execute a SELECT query on the HOURLY_WEATHER table

with open('6_hurricane.sql', 'r') as file:
    query = file.read()

result = engine.execute(query)

for row in result:
    print(row)

In [ ]:
engine.execute(QUERY_1).fetchall()

In [ ]:
write_query_to_file(QUERY_1, QUERY_1_FILENAME)

## Part 4: Visualizing the Data

### Visualization 1

In [ ]:
def plot_frequency_hour(dataframe: pd.DataFrame):
    figure, axes = plt.subplots(figsize=(20, 10))
    
    hour = [1, 2, 3, 4, 5]
    values = [1, 5, 3, 2, 5]

    axes.bar(hour, values)

    axes.set_ylabel('Popularity')
    axes.set_xlabel('Hour')
    axes.set_title("Frequency per Hour")
    axes.set_xlim(-1, 11)
    axes.set_ylim(-1.5, 1.5)
    
    plt.show()

In [ ]:
def get_data_frequency_hour():
    raise NotImplementedError()

In [ ]:
some_dataframe = get_data_frequency_hour()
plot_frequency_hour(some_dataframe)

### Visualization 2

In [521]:
def plot_avg_distance_month(dataframe: pd.DataFrame):
    figure, axes = plt.subplots(figsize=(30, 20))
    
    month = ["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"]
    distance = [1, 5, 3, 2, 5, 7, 8, 1, 9, 23, 6, 7]

    x = np.linspace(0, 12, 12) 

    a, b = np.polyfit(x, distance, deg=1)
    y_est = a * x + b
    y_err = x.std() * np.sqrt(1/len(x) +
                            (x - x.mean())**2 / np.sum((x - x.mean())**2))

    axes.plot(x, y_est, '-')

    axes.plot(month, distance, 'o', color='tab:brown')
    axes.fill_between(x, y_est - y_err, y_est + y_err, alpha=0.2)
    axes.set_ylabel('Average Distance')
    axes.set_xlabel('Month')
    axes.set_title("Average Distance per Month")

    plt.show()

In [ ]:
def get_data_avg_distance_month():
    raise NotImplementedError()

In [ ]:
some_dataframe = get_data_avg_distance_month()
plot_avg_distance_month(some_dataframe)

### Visualization 3

In [ ]:
def plot_dropoffs_ny_area(dataframe: pd.DataFrame):

    days = ['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']

    lga = [4, 5, 1, 8, 3, 9, 2]
    jfk = [9, 2, 8, 4, 5, 8, 1]
    ewr = [9, 7, 3, 7, 2, 9, 4]

    x_axis = np.arange(len(days))
  
    plt.bar(x_axis - 0.2, lga, 0.4, label = 'LGA')
    plt.bar(x_axis + 0.2, jfk, 0.4, label = 'JFK')
    plt.bar(x_axis + 0.4, ewr, 0.4, label = 'EWR')

    plt.xlabel("Days in Week")
    plt.ylabel("Drop Offs")
    plt.title("Drop-Offs per Airport")

    plt.xticks(x_axis, days)
    plt.tight_layout()

    plt.legend()
    plt.show()

In [ ]:
def get_data_dropoffs_ny_area():
    raise NotImplementedError()

In [ ]:
some_dataframe = get_data_dropoffs_ny_area()
plot_dropoffs_ny_area(some_dataframe)

### Visualization 4

In [ ]:
def plot_trips_area(dataframe: pd.DataFrame):    
    map_obj = folium.Map(location = [40.730610, -73.935242], zoom_start = 10, min_zoom = 10, tiles='CartoDB positron')

    lats_longs = [
                    [40.7554, -73.9862],
                    [40.7794, -73.9654],
                    [40.7223, -73.9982],
                    [40.7455, -74.0071],
                ]

    HeatMap(lats_longs).add_to(map_obj)
    return map_obj

In [ ]:
def get_data_trips_area():
    raise NotImplementedError()

In [ ]:
some_dataframe = get_data_trips_area()
plot_trips_area(some_dataframe)

### Visualization 5

In [524]:
def plot_tips_distance(dataframe: pd.DataFrame):
    figure, axes = plt.subplots(figsize=(20, 10))
    
    tips = [2, 6, 3, 7, 8, 1, 9, 22, 9, 6, 22, 1]
    distance = [1, 5, 3, 2, 5, 7, 8, 1, 9, 23, 6, 7]

    axes.scatter(distance, tips, marker='o', alpha=0.5)
    axes.set_title("Yellow Tips - Tips vs. Distance")
    axes.set_ylabel('Popularity')
    axes.set_xlabel('Distance')
    axes.set_xlim(-1, 11)
    axes.set_ylim(-1, 10)
    
    plt.show()

In [ ]:
def get_data_tips_distance():
    raise NotImplementedError()

In [ ]:
some_dataframe = get_data_tips_distance()
plot_tips_distance(some_dataframe)

### Visualization 6

In [ ]:
%matplotlib notebook

In [505]:
def plot_tips_precipitation(dataframe: pd.DataFrame):
    figure, axes = plt.subplots(figsize=(20, 10))
    
    tips = [2, 6, 3, 7, 8, 1, 9, 22, 9, 6, 22, 1]
    precipication = [1, 5, 3, 2, 5, 7, 8, 1, 9, 23, 6, 7]

    ## Animation

    frames = 10
    points = len(tips)
    np.random.seed(42)

    sizes = itertools.cycle([10, 50, 150])
    colors = np.random.rand(frames, points)
    colormaps = itertools.cycle(['Purples', 'Blues', 'Greens', 'Oranges', 'Reds'])
    markers = itertools.cycle(['o', 'v', '^', 's', 'p'])

    def update(i):
        axes.clear()

        axes.scatter(precipication, tips,
                s=next(sizes),
                c=colors[i, :],
                cmap=next(colormaps),
                marker=next(markers),
                alpha=0.5)

        axes.set_title("Yellow Taxi - Tips vs. Precipitation")
        axes.set_ylabel('Precipitation')
        axes.set_xlabel('Tips')
        axes.set_xlim(-1, 11)
        axes.set_ylim(-1, 10)

    anim = animation.FuncAnimation(figure, update, frames=frames, interval=500)
    anim
    plt.show()

In [ ]:
def get_data_tips_precipitation():
    raise NotImplementedError()

In [ ]:
some_dataframe = get_data_tips_precipitation()
plot_tips_precipitation(some_dataframe)